In [225]:
fxobj_raw = """
        move.l  #$00008000,col1(a6)
        move.l  #$00C00000,dstoffx(a6)
        move.l  #$00C00000,dstoffy(a6)
        clr.l   dstoffz(a6)
        move.l  #$00000000,phase1(a6)
        move.l  #$00000400,phase2(a6)
        move.l  #$00140000,i(a6)
        move.l  #$00000400,j(a6)
        move.l  #$00000400,k(a6)
        move.l  #$00000400,zamp(a6)
        move.l  #$00000003,asym_fla(a6)
        move.l  #$001E7540,sine_bas(a6)
        move.l  #$00200000,dy(a6)
        move.l  #$00000008,rsym_ord(a6)
        move.l  #$00080000,rsym_ste(a6)
        move.l  #$00000000,rsym_ist(a6)
        move.l  #$00C00000,rxcen(a6)
        move.l  #$00C00000,rycen(a6)
        move.l  #$00000800,roscale(a6)
        move.l  #$00000000,roscal2(a6)
        move.l  #$00000000,roscalei(a6)
        move.l  #$00000000,drxcen(a6)
        move.l  #$00000000,drycen(a6)
        move.l  #$00001000,cvx(a6)
        move.l  #$00020000,cvy(a6)
        move.l  #$00080000,radius(a6)
        move.l  #$00008000,_i1(a6)
        move.l  #$00008000,_i2(a6)
        move.l  #$00008000,_i3(a6)
        move.l  #$00008000,_i4(a6)
        move.l  #$00008000,_i5(a6)
        move.l  #$00008000,_i6(a6)
        move.l  #$001E7540,wave_2(a6)
        move.l  #$00000000,phase4(a6)
        move.l  #$00021555,phase5(a6)
        move.l  #$00000000,plot_mod(a6)
        move.l  #$01850000,vfb_xsca(a6)
        move.l  #$01850000,vfb_ysca(a6)
        move.l  #$00000000,vfb_angl(a6)
        move.l  #$00C00000,$18(a6)
        move.l  #$00C00000,$1C(a6)
        move.l  #$00780000,$20(a6)
        move.l  #$00000000,$4(a6)
        move.l  #$00E00000,$8(a6)
        move.l  #$00C00000,vfb_xpos(a6)
        move.l  #$00C00000,vfb_ypos(a6)
        move.l  #$0008F000,colx(a6)
        move.l  #$0008F000,coly(a6)
        move.l  #$00010000,cvx2(a6)
        move.l  #$00010000,cvy2(a6)
        move.l  #$00000000,pixsize(a6)
        move.l  #$00400000,phase4(a6)
        clr.l   thang(a6)
        move.l  #$00000000,info(a6)
        clr.l   height(a6)
        clr.l   _mtrig(a6)
"""

In [132]:
vlm_equs = [x for x in open("../src/vlm/vlm.inc").read().split('\n') if ";" not in x][:-1]

vlm_vars = {}
for v in vlm_equs:
    xs = v.split("EQU")
    k = xs[0].strip()
    v = int(xs[1].strip()[1:],16)
    vlm_vars[k] = v
    

## Create the fx_obj base

In [226]:
def split_bytes(b):
    bs = []
    for i in range(2,10,2):
        bs += [b[i:i+2]]
    return bs

fx_as = [x.strip()[8:] for x in fxobj_raw.split('\n')[1:-1]]
fxobj_base = [('name',i,['00']*4) for i in range(0,256,4)]

for l in fx_as:
    xs = l.split(',')
    v = split_bytes(xs[0]) if len(xs) > 1 else ['00'] * 4
    o = xs[1].split('(')[0] if len(xs) > 1 else xs[0].split('(')[0]
    offset = vlm_vars[o] if o in vlm_vars else int(o[1:],16)
    offset = int(offset/4)
    fxobj_base[offset] = (o,offset*4,v)

fxobj_base += [('name',i,['FF']*4) for i in range(256,512,4)]
fxobj_base += [('name',i,['00']*4) for i in range(512,768,4)]

osc_raw = """
oscbank:        dc.l $0400000
                dc.l $0011100, $001E7540
                dc.l $0000000, $0000000
                dc.l $0016300, $001E7540
                dc.l $0000000, $0000000
                dc.l $0003000, $001E7540
                dc.l $0000000, $0000000
                dc.l $000C000, $001E7540
                dc.l $0000000, $0000000
                dc.l $0010000, $001E7540
                dc.l $0000000, $0000000
                dc.l $000E000, $001E7540
                dc.l $0000000, $0000000
                dc.l $000C300, $001E7540
                dc.l $0000000, $0000000
                dc.l $0000F310, $001E7540, $00000000
"""
OFFSET = 768
osc_string = ','.join([x[21:] for x in osc_raw.split('\n')[1:-1]])
osc_vals = ['#$'+x.strip()[1:].rjust(8,'0') for x in osc_string.split(',')]
fxobj_base += [('oscbank',(i*4)+OFFSET,split_bytes(x)) for i,x in enumerate(osc_vals)]

pixcon_raw = """
pixcon:         dc.l $00800000, $00000000
                dc.l $00001600, $00003000, $00FFFF00, $00000000
                dc.l $00040000
"""
OFFSET = 896
osc_string = ','.join([x[21:] for x in pixcon_raw.split('\n')[1:-1]])
osc_vals = ['#$'+x.strip()[1:].rjust(8,'0') for x in osc_string.split(',')]
fxobj_base += [('pixcon',(i*4)+OFFSET,split_bytes(x)) for i,x in enumerate(osc_vals)]

piycon_raw = """
piycon:         dc.l $00800000, $00000000
                dc.l $00001600, $00003000, $00FFFF00, $00000000
                dc.l $00040000
"""
OFFSET = 896+28
osc_string = ','.join([x[21:] for x in piycon_raw.split('\n')[1:-1]])
osc_vals = ['#$'+x.strip()[1:].rjust(8,'0') for x in osc_string.split(',')]
fxobj_base += [('piycon',(i*4)+OFFSET,split_bytes(x)) for i,x in enumerate(osc_vals)]

fxobj_base += [
     ('adsra', 952, ['0A', '00', '01', '00', 'C0', '00', '05', '00', '00', '04']),
     ('adsrb', 962, ['0B', '00', '01', '00', 'C0', '00', '05', '00', '00', '04']),
     ('adsrb', 972, ['0C', '00', '02', '00', 'C0', '00', '05', '00', '00', '04']),
     ('name', 982, ['00', '00']),
     ('py', 984, ['00', '00', '00', '00']),
     ('px', 988, ['00', '00', '00', '00']),
     ('delayf', 992, ['00', '00']),
     ('delayp', 994, ['00', '00']),
     ('dline', 996, ['00', '00', '00', '00']),
     ('delayt', 1000, ['00', '00']),
     ('delayn', 1002, ['00', '00']),
]

fxobj_base

[('name', 0, ['00', '00', '00', '00']),
 ('$4', 4, ['00', '00', '00', '00']),
 ('$8', 8, ['00', 'E0', '00', '00']),
 ('vfb_xsca', 12, ['01', '85', '00', '00']),
 ('vfb_ysca', 16, ['01', '85', '00', '00']),
 ('vfb_angl', 20, ['00', '00', '00', '00']),
 ('$18', 24, ['00', 'C0', '00', '00']),
 ('$1C', 28, ['00', 'C0', '00', '00']),
 ('$20', 32, ['00', '78', '00', '00']),
 ('dstoffx', 36, ['00', 'C0', '00', '00']),
 ('dstoffy', 40, ['00', 'C0', '00', '00']),
 ('vfb_xpos', 44, ['00', 'C0', '00', '00']),
 ('vfb_ypos', 48, ['00', 'C0', '00', '00']),
 ('dstoffz', 52, ['00', '00', '00', '00']),
 ('name', 56, ['00', '00', '00', '00']),
 ('dy', 60, ['00', '20', '00', '00']),
 ('name', 64, ['00', '00', '00', '00']),
 ('name', 68, ['00', '00', '00', '00']),
 ('rsym_ord', 72, ['00', '00', '00', '08']),
 ('rsym_ste', 76, ['00', '08', '00', '00']),
 ('rsym_ist', 80, ['00', '00', '00', '00']),
 ('_i1', 84, ['00', '00', '80', '00']),
 ('_i2', 88, ['00', '00', '80', '00']),
 ('_i3', 92, ['00', '00', '80'

In [255]:
def patch_effect(bs,start=0):
    offset = 0
    effect = [None] * 6144
    for i in range(start,len(bs),2):
        offset += int(bs[i][1:],16)
        if offset > 6144:
            break
        val = bs[i+1][1:]
        effect[offset] = val
        offset += 1 
        if offset > 6144:
            break
    return (i,effect)

bank_data = open("../src/vlm/banks.s",'r').read().splitlines()
bank_bytes = ','.join([l[6:37] for l in bank_data[68:783]]).split(',')
print(bank_bytes)

offset = 0
effect_patches = []
for i in range(0,9):
    offset,effect_patch = patch_effect(bank_bytes,offset)
    effect_patches += [effect_patch]


['$01', '$14', '$00', '$80', '$01', '$01', '$00', '$48', '$00', '$38', '$01', '$01', '$00', '$12', '$00', '$20', '$0F', '$FC', '$03', '$78', '$02', '$5E', '$00', '$EB', '$00', '$FF', '$5D', '$00', '$00', '$65', '$00', '$40', '$0B', '$01', '$60', '$65', '$00', '$21', '$00', '$93', '$00', '$02', '$07', '$04', '$01', '$01', '$01', '$00', '$1A', '$80', '$00', '$00', '$02', '$80', '$00', '$00', '$DF', '$FF', '$19', '$EF', '$00', '$28', '$02', '$B9', '$00', '$DF', '$E0', '$00', '$00', '$32', '$00', '$FA', '$00', '$B1', '$0D', '$8E', '$00', '$DC', '$00', '$43', '$0D', '$35', '$00', '$EE', '$00', '$30', '$0D', '$D7', '$00', '$B8', '$00', '$C0', '$0D', '$1F', '$00', '$A1', '$0E', '$FB', '$00', '$AC', '$00', '$E0', '$0D', '$DB', '$00', '$17', '$00', '$A3', '$0D', '$11', '$00', '$17', '$00', '$CD', '$00', '$10', '$6F', '$90', '$01', '$1B', '$00', '$F3', '$00', '$F0', '$17', '$00', '$FF', '$00', '$FF', '$FF', '$FF', '$00', '$E0', '$FF', '$00', '$FF', '$03', '$01', '$00', '$78', '$00', '$C0', '$17'

In [257]:
def create_effect_objects(effect):
    sub_effects = []
    fx_obj_template = fxobj_base.copy()
    for p in range(0,6144,1024):
        fx_obj = fx_obj_template.copy()
        print(fx_obj)
        for e,(n,i,bs) in enumerate(fx_obj):
            lb = len(bs)
            ns = bs.copy()
            for k,j in enumerate(range(i,i+lb)):
                if effect[p+j]:
                    ns[k] = effect[p+j]
            fx_obj[e] = (n,i,bs,ns)
        sub_effects += [fx_obj]
        fx_obj_template = fx_obj.copy()
    return sub_effects

sub_effects = create_effect_objects(effect_patches[2])
for i,sub_effect in enumerate(sub_effects):
    print(f"; Sub-Effect {i}")
    for nm,offset,_,bs in sub_effect:
        if nm == "info":
            print(f"; dc.l ${''.join(bs)}  ; Byte {offset}-{offset+len(bs)}: {nm}")

[('name', 0, ['00', '00', '00', '00']), ('$4', 4, ['00', '00', '00', '00']), ('$8', 8, ['00', 'E0', '00', '00']), ('vfb_xsca', 12, ['01', '85', '00', '00']), ('vfb_ysca', 16, ['01', '85', '00', '00']), ('vfb_angl', 20, ['00', '00', '00', '00']), ('$18', 24, ['00', 'C0', '00', '00']), ('$1C', 28, ['00', 'C0', '00', '00']), ('$20', 32, ['00', '78', '00', '00']), ('dstoffx', 36, ['00', 'C0', '00', '00']), ('dstoffy', 40, ['00', 'C0', '00', '00']), ('vfb_xpos', 44, ['00', 'C0', '00', '00']), ('vfb_ypos', 48, ['00', 'C0', '00', '00']), ('dstoffz', 52, ['00', '00', '00', '00']), ('name', 56, ['00', '00', '00', '00']), ('dy', 60, ['00', '20', '00', '00']), ('name', 64, ['00', '00', '00', '00']), ('name', 68, ['00', '00', '00', '00']), ('rsym_ord', 72, ['00', '00', '00', '08']), ('rsym_ste', 76, ['00', '08', '00', '00']), ('rsym_ist', 80, ['00', '00', '00', '00']), ('_i1', 84, ['00', '00', '80', '00']), ('_i2', 88, ['00', '00', '80', '00']), ('_i3', 92, ['00', '00', '80', '00']), ('_i4', 96, [

ValueError: too many values to unpack (expected 3)